In [1]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

# Tickers list
ticker_list = ["^DJI", "AA", "AIG", "AXP", "BA", "C", "CAT", "DD", "DIS", "GE",
               "GM", "HD", "HON", "HPQ", "IBM", "INTC", "JNJ", "JPM", "KO", "MCD",
               "MMM", "MO", "MRK", "MSFT", "PFE", "PG", "T", "UTX", "VZ", "WMT", "XOM"]

# Dates
today = date.today()
start_date = '2016-1-1'
end_date = '2020-1-1'

def SaveData(df, filename):
    df.to_csv(r'data/'+filename+'.csv')
    
def GetData(ticker_list):
    df=[]
    for ticker in ticker_list:
        data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date, progress=False)
        data['Symbol'] = ticker 
        df.append(data)
    print("Download Complete")
    return df

# Get the Data
df = GetData(ticker_list)
df = pd.concat(df)
df = df.reset_index()
df = df[['Date', 'Adj Close', 'Symbol']]
stocks = df.pivot('Date','Symbol','Adj Close').reset_index()

Download Complete


In [2]:
# Useful data
returns = np.log(stocks[ticker_list]/stocks[ticker_list].shift(1))
returns.insert(0,'Date',stocks['Date'])
returns = returns.drop([0],axis=0)

returns.set_index('Date', inplace=True)
returns.index = pd.to_datetime(returns.index)
monthly_returns = returns.resample('1M').ffill()

# Risk-free Treasury bills 13 months 
risk_free_rate = 0.029

# Computing betas
beta = []
for ticker in ticker_list:
    x = monthly_returns[ticker_list[0]]
    y = monthly_returns[ticker]
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    beta.append(round(slope,2))

beta = pd.DataFrame(beta, columns={'Beta'})
beta.index = ticker_list
beta['Return'] = round(returns.mean()*252*100,2)

SaveData(beta,'beta')